# 🏡 House Price Prediction with Detailed Exploratory Analysis 🏘️

Thank you for checking out the kernel 🌽


## Import data

In [ ]:
import numpy as np # stats
import pandas as pd # data manipulation
import matplotlib.pyplot as plt # graph
import seaborn as sns # graph
from warnings import filterwarnings

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

print('The train set has {} rows, {} columns'.format(*train.shape))
print('The test set has {} rows, {} columns'.format(*test.shape))


Combine both train and test set into one, then drop the 'Id' column

In [ ]:
all_data = pd.concat([train, test], sort = False, ignore_index = True)
all_data.drop('Id', axis = 1, inplace = True)
print('The all set has {} rows, {} columns'.format(*all_data.shape))

## Understand the target variable: SalePrice
- Ranges using the pandas `describe()`
- Distribution using histogram
- Normality using histogram, qqplot, and statistics: skewness, shapiro etc

In [ ]:
from scipy.stats import norm, skew, probplot, shapiro, normaltest, boxcox
from scipy.special import boxcox1p, inv_boxcox1p

sns.set() # sns default theme
plt.rcParams['figure.figsize'] = [15, 5]
plt.subplots_adjust(wspace = 0.5)
def check_normality(x):
    m, s = norm.fit(x)
    plt.figure()
    plt.subplot(1, 2, 1)
    sns.distplot(x, fit=norm)
    plt.legend(['Normal dist\nμ = {:.2f}\nσ = {:.2f}'.format(m, s)], loc='best', )
    plt.subplot(1, 2, 2)
    probplot(x, plot = plt)
    sk = skew(x)
    sh = shapiro(x)
    print('''
    skew: {:.3f}
    Shapiro-Wilk Test: Statistics = {:.3f}, p = {:.3e}
    D’Agostino’s K^2 Test: Statistics = {:.3f}, p = {:.3e}
    '''.format(skew(x), *shapiro(x), *normaltest(x)))

print(train.SalePrice.describe())
check_normality(train.SalePrice)

Our orignal SalePrice is highly skewed. We can reduce non-normality by:
- Log transform
- Boxcox transform


In [ ]:
check_normality(np.log1p(train.SalePrice))

tmp, maxlog = boxcox(train.SalePrice) # maxlog is the lambda that maximize the log-likelihood function
check_normality(tmp)

Since boxcox transform produce a lower absolute skew, we will use boxcox transform for subsequence analysis.

We assgin new variables named **SalePriceBC** (SalePrice boxcox transformed) and **SalePriceL** (SalePrice log transformed) to the all dataset

In [ ]:
all_data = all_data.assign(SalePriceBC = np.append(
    boxcox1p(train.SalePrice, maxlog),
    np.repeat(np.nan, test.shape[0])
))
all_data = all_data.assign(SalePriceL = np.append(
    np.log1p(train.SalePrice),
    np.repeat(np.nan, test.shape[0])
))

## Understand the explanatory variables (features)
- They are described in data_description.txt
- There are 79 of them:
  - 36 are numeric
  - 43 are str
- Of them, 34 contain missing values. Some are only missing in the train set, some are only in the test set, some are in both.
  


In [ ]:
all_data.dtypes.value_counts()
tmp = pd.DataFrame({
    'all': all_data.isnull().sum(),
    'train': train.isnull().sum(),
    'test': test.isnull().sum(),
    'dtype': all_data.dtypes,
}, index = all_data.isnull().sum().index) # need this index otherwise the resulting data.frame orders the index by alphabet
tmp

In [ ]:
tmp.loc[tmp.iloc[:,0:3].any(1).compress(lambda x: x).index.values]

We will go through all these 34 variables one by one 🐢 and determine how to recover (impute) the missing values. Missing values can be imputed using:
- Mean
- Median
- Most common value
- Or fancy package such as MICE

In [ ]:
# a helper function to examine missingness in var with string value
def exam(x, hue = 'Dataset', data = all_data.assign(Dataset = all_data.SalePrice.isnull().replace({ False: 'train', True: 'test' }))):
    data[x].fillna('MISSING', inplace = True)
    plt.figure()
    p = sns.countplot(x = x, hue = hue, data = data)
    for i in p.patches:
        height = i.get_height()
        height = 0 if np.isnan(height) else height
        p.text(i.get_x() + i.get_width() / 2, height + 10, '{:.0f}'.format(height), ha = 'center')
        
# also create a new dataframe so not to mutate the "all"
all1 = all_data.copy()

### MSZoning
- 0 missing in the train set
- 4 missing in the test set
- Change them to the most common: RL (Residential Low Density)

In [ ]:
exam('MSZoning')
all1.MSZoning.fillna('RL', inplace = True)

### LotFrontage
- 259 missing in the train set
- 277 missing in the test set
- Possible to fill them with mean, median or based on *Neighborhood*
- Decided to fill them based on median per *Neighborhood* using the entire dataset (train + test). Not sure if this is right?


In [ ]:
all1.LotFrontage.describe()
all1.LotFrontage = all1.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

### Alley
-  2721 missing in the train set
-  1639 missing in the test set
- Change them to *None*

In [ ]:
exam('Alley')
all1.Alley.fillna('None', inplace = True)

### Utilities
-  0 missing in train set
-  2 missing in test set

Most of them are AllPub: All public Utilities (E,G,W,& S). Change the missing to AllPub. The lone 1 observation of NoSeWa is strange! This variable is not likely to help with predictive model, possible to drop it

In [ ]:
exam('Utilities')
all1.Utilities.fillna('AllPub', inplace = True)

### Exterior1st & Exterior2nd
-  0 missing in the train set
-  1 missing in the test set
- The exterior material is probably related to RoofStyle and RoofMatl. The missing observation has RoofStyle of Flat and RoofMatl of Tar&Grv. Houses with these roofs have exterior as Plywood. So change the missing value to *Plywood*

In [ ]:
exam('Exterior1st')
exam('Exterior2nd')

all1[all1.Exterior1st.isnull()]
all1[all1.RoofStyle == 'Flat'].Exterior1st.value_counts()
all1[all1.RoofStyle == 'Flat'].Exterior2nd.value_counts()
all1[all1.RoofMatl == 'Tar&Grv'].Exterior1st.value_counts()
all1[all1.RoofMatl == 'Tar&Grv'].Exterior2nd.value_counts()

all1.Exterior1st.fillna('Plywood', inplace = True)
all1.Exterior2nd.fillna('Plywood', inplace = True)

### MasVnrType & MasVnrArea
-  8 missing in the train set
-  16 missing in the test set
- They could be related to LotShape, Neighborhood, Neighborhood. But as the most common is None, so change them all to *None* for MasVnrType, and to *0* for MasVnrArea

In [ ]:
exam('MasVnrType')
all1.MasVnrType.fillna('None', inplace = True)
all1.MasVnrArea.fillna(0, inplace = True)

### Basement: BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath, BsmtHalfBath 
- 37+ missing in train set
- 42+ missing in test set
- The decription indicates NA as No Basement
- A careful examination shows that 9 houses actually have a basement, but unfinished or being restored.
- Examine these 9 one by one and change them accordingly
- Change the rest to *None*

In [ ]:
exam('BsmtQual')
exam('BsmtCond')
exam('BsmtExposure')
exam('BsmtFinType1')
exam('BsmtFinType2')

# these houses below actually have basement
all1[(
    all1.BsmtQual.isnull() |
    all1.BsmtCond.isnull() |
    all1.BsmtExposure.isnull() | 
    all1.BsmtFinType1.isnull() |
    all1.BsmtFinType2.isnull()
) & all1.TotalBsmtSF > 0]

# change these houses accordingly
all1.loc[all1.BsmtQual.isnull() & all1.TotalBsmtSF > 0, 'BsmtQual'] = 'TA' # TA is the most common
all1.loc[all1.BsmtCond.isnull() & all1.TotalBsmtSF > 0, 'BsmtCond'] = 'TA'
all1.loc[all1.BsmtExposure.isnull() & all1.TotalBsmtSF > 0, 'BsmtExposure'] = 'No' # No is the most common
all1.loc[all1.BsmtFinType2.isnull() & all1.TotalBsmtSF > 0, 'BsmtFinType2'] = 'Unf' # Unf is the most common

# the rest filled with None
all1.BsmtQual.fillna('None', inplace = True)
all1.BsmtCond.fillna('None', inplace = True)
all1.BsmtExposure.fillna('None', inplace = True)
all1.BsmtFinType1.fillna('None', inplace = True)
all1.BsmtFinType2.fillna('None', inplace = True)

# or 0 for numeric
all1.BsmtFinSF1.fillna(0, inplace = True)
all1.BsmtFinSF2.fillna(0, inplace = True)
all1.BsmtUnfSF.fillna(0, inplace = True)
all1.TotalBsmtSF.fillna(0, inplace = True)
all1.BsmtFullBath.fillna(0, inplace = True)
all1.BsmtHalfBath.fillna(0, inplace = True)

### Electrical
-  1 missing in the train set
-  0 missing in the test set
- Change them to *SBrkr* (the most common)

In [ ]:
exam('Electrical')
all1.Electrical.fillna('SBrkr', inplace = True)

### KitchenQual
-  0 missing in the train set
-  1 missing in the test set
- Change them to *TA* (the most common)

In [ ]:
exam('KitchenQual')
all1.KitchenQual.fillna('TA', inplace = True)

### Functional
-  0 missing in the train set
-  2 missing in the test set
- Change them to *Typ* (the most common)

In [ ]:
exam('Functional')
all1.Functional.fillna('Typ', inplace = True)

### FireplaceQu
- 690 missing in the train set
- 730 missing in the test set
- Change them to *None*

In [ ]:
exam('FireplaceQu')
all1.FireplaceQu.fillna('None', inplace = True)

### Garage: GarageType, GarageYrBlt, GarageFinish, GarageCars, GarageArea, GarageQual, GarageCond
- ~81 missing in the train set
-  ~78 missing in the test set
- There is an observation where GarageYrBlt is 2207, must be a typo, will change it to 2007, as YearBuilt in 2006
- There are also 18 observations where GarageYrBlt < YearBuilt. Set GarageYrBlt = YearBuilt
- A careful examination found that in the test set, 2 observation actually have a GarageType (Detchd) but are missing in the other columns .
- Change these 2 accordingly.
- For the rest, change GarageType, GarageFinish, GarageQual and GarageCond to *None*, GarageCars and GarageArea to *0*, GarageYrBlt to be the same as median of GarageYrBlt of houes with the same *YearBuilt*, if no median, fill GarageYrBlt with YearBuilt
- The first GarageYrBlt was 1895 (Detchd) in observation 2217 (the first true automobile was thought to be invented in 1885), interesting.

In [ ]:
all1.GarageYrBlt.describe()

all1.GarageYrBlt.sort_values(ascending = False).index[0] # observation: 2592
all1.loc[2592, ['GarageYrBlt', 'YearBuilt', 'YearRemodAdd', 'YrSold']]

# change to 2007
all1.loc[2592, 'GarageYrBlt'] = 2007
all1.loc[2592, ['GarageYrBlt', 'YearBuilt', 'YearRemodAdd', 'YrSold']]

sns.scatterplot(x = 'YearBuilt', y = 'GarageYrBlt', data = all1)
tmp = all1.GarageYrBlt < all1.YearBuilt # those where a garage was built before the house
all1.loc[tmp, ['GarageYrBlt', 'YearBuilt']]
all1.loc[tmp, 'GarageYrBlt'] = all1.loc[tmp, 'YearBuilt']

In [ ]:
# all1[all1.GarageType != 'None'].GarageYrBlt.sort_values()
# all1.loc[2217]

In [ ]:
exam('GarageType')
exam('GarageFinish')
exam('GarageQual')
exam('GarageCond')

# 2 observations with a garage
all1.loc[all1.GarageType.notnull() & all1.GarageCond.isnull(), 'GarageYrBlt'] = all1.loc[all1.GarageType == 'Detchd', 'GarageYrBlt'].median() # 1962, the median of Detchd type
all1.loc[all1.GarageType.notnull() & all1.GarageCond.isnull(), 'GarageFinish'] = 'Unf' # Unf is the most common
all1.loc[all1.GarageType.notnull() & all1.GarageCond.isnull(), 'GarageQual'] = 'TA' # TA is the most common
all1.loc[all1.GarageType.notnull() & all1.GarageCond.isnull(), 'GarageCond'] = 'Unf' # TA is the most common
all1.loc[all1.GarageType.notnull() & all1.GarageCars.isnull(), 'GarageCars'] = all1.loc[all1.GarageType == 'Detchd', 'GarageCars'].median()
all1.loc[all1.GarageType.notnull() & all1.GarageArea.isnull(), 'GarageArea'] = all1.loc[all1.GarageType == 'Detchd', 'GarageArea'].median()

# the rest
all1.GarageType.fillna('None', inplace = True)
all1.GarageCars.fillna(0, inplace = True)
all1.GarageArea.fillna(0, inplace = True)
all1.GarageFinish.fillna('None', inplace = True)
all1.GarageQual.fillna('None', inplace = True)
all1.GarageCond.fillna('None', inplace = True)

all1.GarageYrBlt = all1.groupby('YearBuilt').GarageYrBlt.transform(lambda x: x.fillna(x.median()))
tmp = all1.GarageYrBlt.isnull()
all1.loc[tmp, 'GarageYrBlt'] = all1.loc[tmp, 'YearBuilt']

### PoolQC 
- 1453 missing in the train set
- 1456 missing in the test set
- Change them all to *None*

In [ ]:
exam('PoolQC')
all1.PoolQC.fillna('None', inplace = True)

### Fence
- 1179 missing in the train set
- 1169 missing in the test set
- Change them all to *None*

In [ ]:
exam('Fence')
all1.Fence.fillna('None', inplace = True)

### MiscFeature 
- 1406 missing in the train set
- 1408 missing in the test set
- Change them all to *None*
- Most of the houses have no additional feature, Shed is probably just Gar2, 1 house has a tennis court 🎾, fancy ✨

In [ ]:
exam('MiscFeature')
all1.MiscFeature.fillna('None', inplace = True)

### SaleType 
- 0 missing in the train set
- 1 missing in the test set
- Change them *WD*
- There is one observation in the test set that the house was sold before it was built, YearBuilt > YrSold

In [ ]:
exam('SaleType')
all1.SaleType.fillna('WD', inplace = True)

In [ ]:
exam('SaleCondition')
all1[all1.YearBuilt > all1.YrSold] # this house SaleCondition was Partial

### Now there should not be any missing in the train and test dataset

In [ ]:
all1.loc[:, ~all1.columns.isin(['SalePrice', 'SalePriceBC'])].isnull().any().any()

## Understand the relationship between the target variable (SalePrice) and explanatory variables
- Compute correlation matrix
- Visualise using a heatmap
- The most correlated variables to SalePrice are:
  - OverallQual
  - GrLivArea
  - GarageCars
  - GarageArea
  - TotalBsmtSF
These make sense as the bigger the house the more expensive it is
  


In [ ]:
tmp = all1[all1.SalePrice.notnull()]
tmp_corr = tmp.corr()

plt.figure(figsize = (15, 9))
sns.heatmap(tmp_corr, square = True)

tmp_corr.SalePrice.sort_values(ascending = False)

### Outliers in GrLivArea
- There are two observations with a large living area (>4000) but were sold relatively cheaply (<200,000). They are true outliers (also according to http://ww2.amstat.org/publications/jse/v19n3/Decock/DataDocumentation.txt)
- It's not ideal to remove data, but in this case, it's justfied because we are interested in predicting prices for **typical** houses.
- I will create a new variable named **IsOutlier** and then later on we can compare models where outliers are removed vs models where outliers are not removed.

In [ ]:
plt.figure(figsize = (12, 9))
sns.scatterplot(x = 'GrLivArea', y = 'SalePrice', data = all1)

all1 = all1.assign(IsOutlier = np.where((all1['GrLivArea'] > 4000) & (all1['SalePrice'] < 200000), 1, 0))

In [ ]:
# Also check other variables

sns.scatterplot(x = 'LotArea', y = 'SalePrice', data = all1); plt.figure()
sns.scatterplot(x = 'MasVnrArea', y = 'SalePrice', data = all1); plt.figure()
sns.scatterplot(x = 'TotalBsmtSF', y = 'SalePrice', data = all1); plt.figure()
sns.scatterplot(x = 'GarageArea', y = 'SalePrice', data = all1)

### OverallQual
- This feature has the highest correlation with SalePrice
- It's worth having a more detailed look
- The higher the quality, the higher the sale price (as expected)

In [ ]:
sns.boxplot(x = 'OverallQual', y = 'SalePrice', color = 'seagreen', data = all1)

## dtypes in predictors
Some of the predictors should be string:
- MSSubClass
- MoSold, at the moment it's from 1 to 12. It shouldn't make sense that Dec is "greater" than Jan

Some string predictors should be numeric:
- OverallQual &  OverallCond are numeric while ExterQual &  ExterCond, BsmtQual & BsmtCond, HeatingQC, KitchenQual, FireplaceQu, GarageQual & GarageCond, PoolQC are string
- Should \*Qual and \*Cond be converted to numeric, if so, how? One-hot, get_dummies, binary or https://github.com/scikit-learn-contrib/categorical-encoding?

### Qual & Cond
Qual and Cond are ranked from Po Poor to Ex Excellent, so it makes senses to convert them to numeric from 1 to 5

In [ ]:
all1.MSSubClass = all1.MSSubClass.astype(str)
all1.MoSold = all1.MoSold.astype(str)

columns = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']
values = { 'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5 }
all1[columns] = all1[columns].replace(values)

def exam2(x):
    plt.figure()
    sns.boxplot(x = x, y = 'SalePrice', data = all1)


### Street, Alley, Land, style
- Street and Alley seem ordinal, so they will be converted to numeric
- LotShape, LandContour, LotConfig, LandSlope do not seem ordinal, so leave them as they are
- Condition1 & Condition2 are not ordinal
- BldgType is not ordinal
- HouseStyle is marginally ordinal, but will leave it as it (replated to GrLivArea)
- RoofStyle, RoofMatl, MasVnrType, Foundation are not ordinal
- BsmtExposure looks ordinal
- Heating is not ordinal
- CentralAir can be made ordinal
- Electrical is not ordinal
- Functional can be made ordinal
- GarageType is not ordinal
- GarageFinish can be made ordinal
- PavedDrive can be made ordinal
- MiscFeature, SaleType, SaleCondition are not ordinal

In [ ]:
exam2('Street'); all1.Street.replace({ 'Grvl': 0, 'Pave': 1 }, inplace = True)
exam2('Alley'); all1.Alley.replace({ 'No': 0, 'Grvl': 1, 'Pave': 2 }, inplace = True)

# these don't seem ordinal
# exam2('LotShape'); exam2('LandContour'); exam2('LotConfig'); exam2('LandSlope')
# exam2('Condition1'); exam2('Condition2')
# exam2('BldgType'); exam2('HouseStyle')
# exam2('RoofStyle'); exam2('RoofMatl'); exam2('MasVnrType'); exam2('Foundation')
# exam2('Heating')

exam2('BsmtExposure'); all1.BsmtExposure.replace({ 'None': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4 }, inplace = True)
values = { 'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6 }
exam2('BsmtFinType1'); all1.BsmtFinType1.replace(values, inplace = True)
exam2('BsmtFinType2'); all1.BsmtFinType2.replace(values, inplace = True)

# exam2('Heating') # not ordinal
exam2('CentralAir'); all1.CentralAir.replace({ 'Y': 1, 'N': 0 }, inplace = True)

# exam2('Electrical') # not ordinal

exam2('Functional'); all1.Functional.replace({ 'Sal': 0, 'Sev': 1, 'Maj2': 2, 'Maj1': 3, 'Mod': 4, 'Min2': 5, 'Min1': 6, 'Typ': 7 }, inplace = True)

# exam2('GarageType') # not ordinal
exam2('GarageFinish'); all1.GarageFinish.replace({ 'None': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3 }, inplace = True)
exam2('PavedDrive'); all1.PavedDrive.replace({ 'N': 0, 'P': 1, 'Y': 2 }, inplace = True)

# exam2('MiscFeature'); exam2('SaleType'); exam2('SaleCondition')

## Deriving new variables/features (feature engineering)
### Area
- We have: LotArea, MasVnrArea, GrLivArea, TotalBsmtSF, GarageArea, PoolArea, 1stFlrSF, 2ndFlrSF, LowQualFinSF, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch
- Add a TotalArea = GrLivArea + TotalBsmtSF + GarageArea + MasVnrArea 
- Now, correlation between TotalArea & SalePrice is 0.8598, much higher than individual areas alone.
- I have tried adding Porches and/or Deck into TotalArea, but the resulting correlations decreased, so I did not add them to the TotalArea
- Note: adding areas together to derive TotalArea will cause some multi-collinearity. Later in the model buildings, we may need to drop some variables.

In [ ]:
# GrLivArea is usually the sum of 1stFlrSF and 2ndFlrSF, except a few 
tmp = all1[['1stFlrSF', '2ndFlrSF']].apply(sum, axis = 1) == all1.GrLivArea
all1.loc[tmp == 0, ['GrLivArea', '1stFlrSF', '2ndFlrSF']]

# Assign TotalArea
all1 = all1.assign(TotalArea = all1[['GrLivArea', 'TotalBsmtSF', 'GarageArea', 'MasVnrArea']].apply(sum, axis = 1))
all1[['TotalArea', 'GrLivArea', 'TotalBsmtSF', 'GarageArea', 'MasVnrArea', 'SalePrice']].corr()

### Bathroom
- We have: BsmtFullBath, BsmtHalfBath, FullBath, HalfBath
- Add a Bath = BsmtFullBath  + BsmtHalfBath / 2+ FullBath + HalfBath / 2
- Now correlation between Bath & SalePrice is 0.6359

In [ ]:
all1 = all1.assign(Bath = all1.BsmtFullBath + all1.BsmtHalfBath / 2 + all1.FullBath + all1.HalfBath / 2)
all1[['Bath', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'SalePrice']].corr()

### House age & remodelling
- YearBuilt, YearRemodAdd, YrSold
- From these variables, we can determine:
 - The age of the house: YrSold - YearBuilt
 - Has the house ever been remodelled: YearBuilt != YearRemodAdd
 - ~~Was the house new when sold: YrSold == YearBuilt~~
- We see that:
 - There is a slight downward trend for SalePrice the older the house
 - Remodelled houses were slightly of lower prices (probably also remodelled houses were older, smaller?)
 - New houses were much higher in prices

In [ ]:
all1 = all1.assign(Age = all1.YrSold - all1.YearBuilt)
all1.loc[all1.Age < 0, 'Age'] = 0 # There was one house where YrSold < YearBuilt

all1 = all1.assign(IsRemodelled = np.where(all1.YearBuilt == all1.YearRemodAdd, 0, 1))

# all1 = all1.assign(IsNew = np.where(all1.YrSold <= all1.YearBuilt, 1, 0)) # use <= because there was one house where YrSold < YearBuilt
# all1.loc[all1.SaleType == 'New', 'IsNew'] = 1 # some where 

all1.Age.describe()

In [ ]:
sns.distplot(all1.Age, bins = 50); plt.figure()
sns.scatterplot(x = 'Age', y = 'SalePrice', data = all1); plt.figure()
sns.boxplot(x = 'IsRemodelled', y = 'SalePrice', data = all1); plt.figure()
sns.boxplot(x = 'SaleType', y = 'SalePrice', data = all1); plt.figure()

## Skewness in numeric predictors
- We now have 55 numeric variables,  33 of them have skewness more than 0.75 
- We can reduce skewness using log1p or boxcox for some of them
- It appears that boxcox reduce skewness more than log1p

In [ ]:
filterwarnings('ignore')
tmp = all1.skew().compress(lambda x: abs(x) > 0.75).sort_values()
tmp = tmp[tmp.index != 'MSSubClass']
tmp = pd.DataFrame({
    'ori': tmp,
    'after with log': [skew(np.log1p(all1[x]), nan_policy = 'omit') for x in tmp.index],
    'after with boxcox': [skew(boxcox(all1[x] + 1)[0], nan_policy = 'omit') for x in tmp.index]
})
filterwarnings('ignore')
tmp

In [ ]:
for v in ['Functional', 'BsmtCond', 'GarageQual', 'PavedDrive', 'BsmtQual', 'TotRmsAbvGrd', 'ExterQual', '2ndFlrSF', 'BsmtUnfSF', 'BsmtExposure', 'TotalBsmtSF', 'ExterCond', 'BsmtFinSF1', 'TotalArea', 'LotFrontage', 'WoodDeckSF', 'OpenPorchSF', 'MasVnrArea', 'BsmtFinType2', 'ScreenPorch', 'EnclosedPorch', 'BsmtFinSF2', 'KitchenAbvGr', '3SsnPorch', 'LowQualFinSF', 'PoolArea', 'PoolQC', 'MiscVal']:
    all1[v] = boxcox(all1[v] + 1)[0]

for v in ['GrLivArea', '1stFlrSF', 'LotArea']:
    all1[v] = np.log1p(all1[v])

In [ ]:
all1.skew().compress(lambda x: abs(x) > 0.75).sort_values()

## Label encoding

- We are now left with 28 variables of type `str`
- We can convert them using LabelEncoder, OneHotEncoder, LabelBinarizer or MultiLabelBinarizer
- In this instance, I am going to use the OneHotEncoder
- The resulting data.frame is now all numeric, containing 301 columns

In [ ]:
all1.dtypes.value_counts()

In [ ]:
from category_encoders import OneHotEncoder, BinaryEncoder
# all1_n = BinaryEncoder().fit_transform(all1) # tried this, but cv is lower than onehot
all1_n = pd.get_dummies(all1) # similar to all1_n = OneHotEncoder().fit_transform(all1)

print(all1_n.shape)
print(all1_n.dtypes.value_counts())

train_index = all1_n.SalePrice.notnull()

# remove the outliers
outliers = all1_n.IsOutlier == 1
X = all1_n.loc[~outliers & train_index, ~all1_n.columns.isin(['SalePrice', 'SalePriceBC', 'SalePriceL', 'IsOutlier'])]
y = all1_n.loc[~outliers & train_index, 'SalePriceBC'] # boxcox transformed SalePrice
yL = all1_n.loc[~outliers & train_index, 'SalePriceL'] # log transformed SalePrice
yO = all1_n.loc[~outliers & train_index, 'SalePrice'] # Original SalePrice

X_test = all1_n.loc[~train_index, ~all1_n.columns.isin(['SalePrice', 'SalePriceBC', 'SalePriceL', 'IsOutlier'])]

## Modelling

### Libraries

In [ ]:
from sklearn.model_selection import KFold, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import mean_squared_error

# score model performance
cv = KFold(n_splits = 5, shuffle = True, random_state = 0)
def score(model, X = X, y = y, des = ''):
    rmse = np.sqrt(-cross_val_score(model, X = X, y = y, scoring = 'neg_mean_squared_error', cv = cv))
    print(des, '\n', *rmse, '\nMean:', rmse.mean(), '\nSD:', rmse.std())
    return(rmse)

### Baseline

If we guess all the SalePrice as the mean, the RMSE are:
- 0.1583 (using boxcox transformed y)
- 0.3996 (using log transformed y)
- 79467.79 (using original y)
Our models need to beat them

### Good old linear regression
- The mean RMSE is 0.1568, not much better than baseline

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, LassoLarsIC, ElasticNet, ElasticNetCV, BayesianRidge

In [ ]:
# baseline, if we guess all y as y.mean()
# np.sqrt(mean_squared_error(y, np.repeat(y.mean(), len(y)))) # boxcox
# np.sqrt(mean_squared_error(yL, np.repeat(yL.mean(), len(yO)))) # log
# np.sqrt(mean_squared_error(yO, np.repeat(yO.mean(), len(yL)))) # original

In [ ]:
model = make_pipeline(RobustScaler(), LinearRegression())
score(model, des = 'Using boxcox transformed SalePrice as y');
score(model, y = yL, des = '\nUsing log transformed SalePrice as y');
# score(model, y = yO, des = '\nUsing original SalePrice as y');

### Ridge
This improved significantly without any optimisation.

In [ ]:
filterwarnings('ignore')

model = make_pipeline(RobustScaler(), Ridge())
score(model, des = 'Using boxcox transformed SalePrice as y');
score(model, y = yL, des = '\nUsing log transformed SalePrice as y');
# score(model, y = yO, des = '\nUsing original SalePrice as y');

filterwarnings('default')

In [ ]:
filterwarnings('ignore')

score(make_pipeline(Ridge()), des = 'No scaling'); # without scaling seems a little better
score(make_pipeline(StandardScaler(), Ridge()), des = '\nUsing StandardScaler'); # standard scaler seems a litte bit worse

filterwarnings('default')

### Lasso
- Find the best alpha using LassoCV
- RMSE is higher when NOT excluding the outliers
- Lasso also perferoms feature selection. From the beginning 168 features of X, it redues to 83.
- The highest positive coef are:
  - GrLivArea
  - OverallQual
  - Neighborhood
- The highest negative coef are:
  - MSZoning
  - Age
  - BsmtFinType

In [ ]:
# Using LassoCV to find the best alpha
filterwarnings('ignore')
model = LassoCV(
    alphas = np.logspace(-5, 0.5, 50),
    random_state = 0,
    cv = cv,
    max_iter = 100000,
    n_jobs = -1
)
model.fit(RobustScaler().fit_transform(X), y)
filterwarnings('default')

alpha = model.alpha_

def plot_rmse(alphas, rmse):    
    tmp = pd.DataFrame({
        'alpha': alphas,
        '-log(alpha)': -np.log(alphas),
        'rmse': rmse,
    })
    sns.scatterplot(x = '-log(alpha)', y = 'rmse', data = tmp)
    tmp = tmp[tmp.rmse == min(tmp.rmse)] # the min
    plt.annotate(
        'alpha: {:.6f}\nscore: {:.6f}'.format(*tmp[['alpha', 'rmse']].iloc[0]),
        xy = (tmp['-log(alpha)'], tmp['rmse'] + 0.001),
        xytext = (7.5, tmp['rmse'] + 0.025),
        arrowprops = dict(arrowstyle = '->', color = 'green')
    )

plot_rmse(model.alphas_, np.sqrt(np.mean(model.mse_path_, 1)))
tmp = pd.DataFrame({
    'coef': model.coef_,
    'var': X.columns.values
})
tmp = tmp.drop(tmp[abs(tmp.coef) < 0.01].index).sort_values('coef')
tmp

In [ ]:
# Using GridSearchCV to find the best alpha
from sklearn.model_selection import GridSearchCV

filterwarnings('ignore')
model = GridSearchCV(
    Lasso(random_state = 0),
    param_grid = { 'alpha': np.logspace(-5, 0, 50) },
    scoring = 'neg_mean_squared_error',
    n_jobs = -1,
    return_train_score = True,
    cv = cv,
)
model.fit(RobustScaler().fit_transform(X), y)
filterwarnings('default')

for i in model.cv_results_: print(i)
plot_rmse(
    model.cv_results_['param_alpha'].data.astype(np.float64),
    np.sqrt(-model.cv_results_['mean_test_score'])
)

alpha = model.best_params_['alpha']

In [ ]:
model = make_pipeline(RobustScaler(), Lasso(alpha = 0.000212, random_state = 0, max_iter = 10000))
score(model, des = 'Using boxcox transformed SalePrice as y');
score(model, y = yL, des = '\nUsing log transformed SalePrice as y');
# score(model, y = yO, des = '\nUsing original SalePrice as y'); # doesn't converge

In [ ]:
# LassoLarsIC
filterwarnings('ignore')
model = make_pipeline(RobustScaler(), LassoLarsIC())
score(model, des = 'Using boxcox transformed SalePrice as y');
score(model, y = yL, des = '\nUsing log transformed SalePrice as y');
# score(model, y = yO, des = '\nUsing original SalePrice as y');
filterwarnings('default')

In [ ]:
# test the effects of NOT excluding the outliers
model = make_pipeline(RobustScaler(), Lasso(alpha = 0.000212, random_state = 0, max_iter = 100000))
score(
    model,
    X = all1_n.loc[train_index, ~all1_n.columns.isin(['SalePrice', 'SalePriceBC', 'SalePriceL', 'IsOutlier'])],
    y = all1_n.loc[train_index, 'SalePriceBC']
);

### Elastic Net
- Use ElasticNetCV to find the best alpha and l1_ratio
- Similar to Lasso, it selected similar vars: GrLivArea, OverallQual, SaleCondition, MSZoning, Age, SaleType

In [ ]:
# # Using ElasticNetCV to find the best alpha and l1_ratio
model = ElasticNetCV(
    random_state = 0, cv = cv, max_iter = 100000, n_jobs = -1,
    alphas = np.logspace(-4, 0, 25),
    l1_ratio = [0.1, .5, .7, .9, .95, .99]
)
filterwarnings('ignore')
model.fit(X, y)
filterwarnings('default')

print('Best alpha: {}\nbest l1_ratio: {}'.format(model.alpha_, model.l1_ratio_))

In [ ]:
model = make_pipeline(RobustScaler(), ElasticNet(alpha = 0.001695, l1_ratio = 0.1, random_state = 0, max_iter = 100000))
score(model, des = 'Using boxcox transformed SalePrice as y');
score(model, y = yL, des = '\nUsing log transformed SalePrice as y');

In [ ]:
pd.DataFrame({
    'Id': test.Id,
    'SalePrice': inv_boxcox1p(model.fit(X, y).predict(X_test), maxlog)
}).to_csv('submission_elastic_net_y.csv', index = False)

pd.DataFrame({
    'Id': test.Id,
    'SalePrice': np.expm1(model.fit(X, yL).predict(X_test))
}).to_csv('submission_elastic_net_yL.csv', index = False)

In [ ]:
# BayesianRidge
model = make_pipeline(RobustScaler(), BayesianRidge())
score(model, des = 'Using boxcox transformed SalePrice as y');
score(model, y = yL, des = '\nUsing log transformed SalePrice as y');

### Random Forest


In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = make_pipeline(RandomForestRegressor(n_estimators = 500, n_jobs = -1))
filterwarnings('ignore')
score(model);
filterwarnings('default')

### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
model = make_pipeline(GradientBoostingRegressor())
filterwarnings('ignore')
score(model);
score(model, y = yL);
filterwarnings('default')

### XGBoost 

In [ ]:
# from xgboost import XGBRegressor
from xgboost.sklearn import XGBRegressor # with scikit-learn wrapper
model = XGBRegressor(
    learning_rate = 0.046777,
    n_estimators = 250,
    max_depth = 4,
    min_child_weight = 1,
    gamma = 0,
    subsample = 0.5,
    colsample_bytree = 0.6,
    reg_alpha = 0.17704,
    reg_lambda = 0.75,
    n_jobs = -1,
)
score(model);
score(model, y = yL);

In [ ]:
pd.DataFrame({
    'Id': test.Id,
    'SalePrice': inv_boxcox1p(model.fit(X, y).predict(X_test), maxlog)
}).to_csv('submission_xgb_y.csv', index = False)

pd.DataFrame({
    'Id': test.Id,
    'SalePrice': np.expm1(model.fit(X, yL).predict(X_test))
}).to_csv('submission_xgb_yL.csv', index = False)

In [ ]:
# tuning XGBRegressor
# # 1, tune n_estimators, best value found: 250, 0.047966
# model = GridSearchCV(
#     XGBRegressor(n_jobs = -1),
#     { 'n_estimators': np.arange(100, 500, 50) },
#     scoring = 'neg_mean_squared_error', return_train_score = True, cv = cv, n_jobs = -1,
# )

# # 2, tune max_depth, best value found: 4, 0.047581
# model = GridSearchCV(
#     XGBRegressor(n_jobs = -1, n_estimators = 250),
#     { 'max_depth': np.arange(1, 9, 1) },
#     scoring = 'neg_mean_squared_error',
#     return_train_score = True,
#     cv = cv,
#     n_jobs = -1,
# )
# # 3, tune min_child_weight, best value found: 2, 0.047548
# model = GridSearchCV(
#     XGBRegressor(n_jobs = -1, n_estimators = 250, max_depth = 4),
#     { 'min_child_weight': np.arange(1, 9, 1) },
#     scoring = 'neg_mean_squared_error',
#     return_train_score = True,
#     cv = cv,
#     n_jobs = -1,
# )
# # 4, tune gamma, best value found: 0, 0.047581
# model = GridSearchCV(
#     XGBRegressor(n_jobs = -1, n_estimators = 250, max_depth = 4, min_child_weight = 1),
#     { 'gamma': np.arange(0, 0.5, 0.1) },
#     scoring = 'neg_mean_squared_error',
#     return_train_score = True,
#     cv = cv,
#     n_jobs = -1,
# )
# # 5, tune subsample & colsample_bytree, best value found: 0.5 & 0.6,  0, 0.046886
# model = GridSearchCV(
#     XGBRegressor(n_jobs = -1, n_estimators = 250, max_depth = 4, min_child_weight = 1, gamma = 0, ),
#     { 'subsample': np.arange(0.5, 1.01,  0.1), 'colsample_bytree': np.arange(0.5, 1.01, 0.1) },
#     scoring = 'neg_mean_squared_error',
#     return_train_score = True,
#     cv = cv,
#     n_jobs = -1,
# )
# # 6, tune alpha & lambda, best value found: 0.17704 & 0.75, 0.046783
# model = GridSearchCV(
#     XGBRegressor(n_jobs = -1, n_estimators = 250, max_depth = 4, min_child_weight = 1, gamma = 0, subsample = 0.5, colsample_bytree = 0.6),
#     { 'reg_alpha': np.logspace(np.log(1e-4), np.log(1.5), base = np.exp(1), num = 10), 'reg_lambda': [.1, .25, .5, .75, .9, .95, .99, 1] },
#     scoring = 'neg_mean_squared_error',
#     return_train_score = True,
#     cv = cv,
#     n_jobs = -1,
# )
# # 7, tune learning_rate, best value found: 0.1, 0.046777
# model = GridSearchCV(
#     XGBRegressor(n_jobs = -1, n_estimators = 250, max_depth = 4, min_child_weight = 1, gamma = 0, subsample = 0.5, colsample_bytree = 0.6, reg_alpha = 0.17704, reg_lambda = 0.75),
#     { 'learning_rate': [0.01, 0.025, 0.05, 0.75, 0.1] },
#     scoring = 'neg_mean_squared_error',
#     return_train_score = True,
#     cv = cv,
#     n_jobs = -1,
# )

In [ ]:
# the script below run a bayesian optimization search for parameters, take many hours

# from timeit import default_timer as timer
# import pickle

# # https://towardsdatascience.com/an-introductory-example-of-bayesian-optimization-in-python-with-hyperopt-aae40fff4ff0
# from hyperopt import hp, fmin, Trials, tpe, STATUS_OK

# # trials = Trials()
# # or
# # with open('xgb_trials.pickle', 'rb') as handle: trials = pickle.load(handle)

# def objective(params):
#     for k in params: params[k] = [params[k]]
#     params['n_estimators'] = [int(params['n_estimators'][0])]
#     params['max_depth'] = [int(params['max_depth'][0])]
#     params['min_child_weight'] = [int(params['min_child_weight'][0])]    
#     model = GridSearchCV(
#         XGBRegressor(n_jobs = -1),
#         params,
#         scoring = 'neg_mean_squared_error',
#         return_train_score = True,
#         cv = cv,
#         n_jobs = -1,
#     )
#     model.fit(X, y) 
#     result = {
#         'loss': np.sqrt(-model.best_score_),
#         'params': model.best_params_,
#         'elapsed': timer() - start_time,
#     }
#     rounded_result = { k: round(v, 6) if type(v) != dict else { k2: round(v2, 6) for k2, v2, in v.items() } for k, v in result.items() }
#     n_trial = len(trials.tids)
#     print('Trial', n_trial, rounded_result)
#     # save the trials every now and then
#     if n_trial % 20 == 0:
#         with open('xgb_trials.pickle', 'wb') as handle:
#             pickle.dump(trials, handle, protocol = pickle.HIGHEST_PROTOCOL)
    
#     result['status'] = STATUS_OK
#     return(result)

# # set up timer & run
# start_time = timer()

# best = fmin(
#     fn = objective,
#     space = {
#         'n_estimators': hp.quniform('n_estimators', 50, 500, 50),
#         'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.5)),
#         'max_depth': hp.quniform('max_depth', 3, 10, 1),
#         'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
#         'gamma': hp.uniform('gamma', 0, 0.4),
#         'subsample': hp.uniform('subsample', 0.6, 0.9),
#         'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 0.9),
#         'reg_alpha': hp.loguniform('reg_alpha', np.log(1e-5), np.log(1)),
#     },
#     algo = tpe.suggest,
#     trials = trials,
#     rstate = np.random.RandomState(0),
#     max_evals = 10000,
# )

### LightGBM

In [ ]:
from lightgbm import LGBMRegressor
model = make_pipeline(LGBMRegressor())
score(model);

I have 📚 a lot from several notebooks (to name a few): [Erik Bruin](https://www.kaggle.com/erikbruin/house-prices-lasso-xgboost-and-a-detailed-eda), [Serigne](https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard).